In [47]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [40]:
response = requests.post(url='https://banner.rowan.edu/reports/reports.pl?task=Section_Tally',
                data= {"term": "202520",
                        "task": "Section_Tally",
                        "coll": "ALL",
                        "dept": "ALL",
                        "subj": "ALL",
                        "ptrm": "ALL",
                        "sess": "ALL",
                        "prof": "ALL",
                        "attr": "ALL",
                        "camp": "ALL",
                        "bldg": "ALL",
                        "Search": "Search",
                        "format": "excel"})
assert(response.status_code == 200), response.status_code

with open('tally.xml', 'wb') as file:
    file.write(response.content)

In [44]:
from lxml import etree
import csv

# Parse the XML file
parser = etree.XMLParser(recover=True)
root = etree.fromstring(response.content, parser=parser)

In [45]:
# Open a CSV file for writing
with open('tally.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    # Iterate through the XML tree and extract data
    for worksheet in root.findall('{urn:schemas-microsoft-com:office:spreadsheet}Worksheet'):
        sheet_name = worksheet.get('{urn:schemas-microsoft-com:office:spreadsheet}Name')
        
        for table in worksheet.findall('{urn:schemas-microsoft-com:office:spreadsheet}Table'):
            for row in table.findall('{urn:schemas-microsoft-com:office:spreadsheet}Row'):
                row_data = [cell.find('{urn:schemas-microsoft-com:office:spreadsheet}Data').text for cell in row.findall('{urn:schemas-microsoft-com:office:spreadsheet}Cell')]
                csvwriter.writerow(row_data)


In [46]:
df = pd.read_csv('tally.csv')

In [47]:
df.head()

,CRN,Subj,Crse,Sect,Part of Term,Session,Title,Prof,Day Beg End Bldg Room (Type),Campus,...,Hrs,Max,MaxResv,LeftResv,Enr,Avail,WaitCap,WaitCount,WaitAvail,Room Cap
0,20237,ACC,03210,1,Full Term ... \n21-JAN to 09-MAY,Day,PRINCIPLES OF ACCTG I,STAFF,MW 1530 1645 BUSN 131 (Class),Main,...,3.0,35,0,0,0,35,5,0,5,40
1,20241,ACC,03210,2,Full Term ... \n21-JAN to 09-MAY,Evening,PRINCIPLES OF ACCTG I,STAFF,M 1830 1945 BUSN 131 (Class)\nM 20...,Main,...,3.0,35,0,0,0,35,5,0,5,40
2,20239,ACC,03210,3,Full Term ... \n21-JAN to 09-MAY,Day,PRINCIPLES OF ACCTG I,STAFF,TR 1100 1215 BUSN 101 (Class),Main,...,3.0,35,0,0,0,35,5,0,5,40
3,20282,ACC,03210,4,Full Term ... \n21-JAN to 09-MAY,Day,PRINCIPLES OF ACCTG IRCB Pre-business student...,"Lin, Tony",TR 1400 1515 BUSN 131 (Class),Main,...,3.0,35,0,0,0,35,5,0,5,40
4,20283,ACC,03210,5,Full Term ... \n21-JAN to 09-MAY,Day,PRINCIPLES OF ACCTG IRCB Pre-business student...,"Lin, Tony",TR 1530 1645 BUSN 101 (Class),Main,...,3.0,35,0,0,0,35,5,0,5,40


In [48]:
from datetime import datetime

current_day = datetime.now().day
current_month = datetime.now().month
current_year = datetime.now().year

print(current_day, current_month, current_year)

2 10 2024


In [53]:
def _get_term_code(term: str):
    TERM_VALUE = ['40', '30', '20', '10']
    TERM_NAME = ['fall', 'summer', 'spring', 'winter']
    """gets rowan term code, {year}{TERM_VALUE}
    Args:
        term: {fall, summer, spring, winter} followed by year
        ex. spring 2024, summer 2025 ...
    """
    _split = term.split()
    assert len(_split) == 2, "invalid term"
    term, year = _split
    assert len(year) == 4 and year.isalnum(), "invalid term year"

    return f"{year}{TERM_VALUE[TERM_NAME.index(term.lower())]}"

In [54]:
term:str ='Spring 2024'
_get_term_code(term)

'202420'

In [2]:
from ratemyprof import RateMyProfScraper
import pandas as pd
from courseclass import CourseClass

rowan = RateMyProfScraper(822)
rowan.find_prof('Jennifer Kay')

{'tDept': 'Computer Science',
 'tSid': '822',
 'institution_name': 'Rowan University',
 'tFname': 'Jennifer',
 'tMiddlename': '',
 'tLname': 'Kay',
 'tid': 49373,
 'tNumRatings': 45,
 'rating_class': 'good',
 'contentType': 'TEACHER',
 'categoryType': 'PROFESSOR',
 'overall_rating': '4.6'}

In [3]:
df = pd.read_csv('tally.csv')
subj = 'CS'
crse = '02630'
classes = df[(df['Subj'] == subj) & (df['Crse'] == crse)]
classes

,CRN,Subj,Crse,Sect,Part of Term,Session,Title,Prof,Day Beg End Bldg Room (Type),Campus,...,Hrs,Max,MaxResv,LeftResv,Enr,Avail,WaitCap,WaitCount,WaitAvail,Room Cap
1063,22310,CS,02630,1,Full Term ... \n21-JAN to 09-MAY,Day,ADV TOPICS IN DATABASE SYSTEMS,"Kniaz, David A",W 1530 1645 ROBSON 306 (Class)\nW ...,Main,...,3.0,30,0,0,0,30,0,0,0,40
1064,22313,CS,02630,2,Full Term ... \n21-JAN to 09-MAY,Day,ADV TOPICS IN DATABASE SYSTEMS,"Myers, Jack F",R 1230 1345 ROBSON 306 (Class)\nR ...,Main,...,3.0,30,0,0,0,30,0,0,0,40


In [4]:
clses: list[CourseClass] = []
for i in range(len(classes)):
    cls = classes.iloc[i]
    prof = rowan.find_prof(cls['Prof'])

    # if proffesor info is found else just put in string
    prof = prof if prof else cls['Prof']
    cls = CourseClass(subj, crse, prof, cls['Day  Beg   End   Bldg Room  (Type)'], cls['Avail'], cls['Max'])

    clses.append(cls)

In [8]:
"spring 2022".split()

['spring', '2022']

In [7]:
clses[1].prof

{'tDept': 'Computer Science',
 'tSid': '822',
 'institution_name': 'Rowan University',
 'tFname': 'Jack',
 'tMiddlename': '',
 'tLname': 'Myers',
 'tid': 1975389,
 'tNumRatings': 36,
 'rating_class': 'good',
 'contentType': 'TEACHER',
 'categoryType': 'PROFESSOR',
 'overall_rating': '4.4'}

In [22]:
res = requests.post(url='https://catalog.rowan.edu/ajax/preview_course.php?', 
                    data={'catoid': "9",
                          'coid': '30522',
                          'display_options': 'a:2:{s:8:~location~;s:8:~template~;s:28:~course_program_display_field~;s:0:~~;}',
                          "show": None,
                        })
assert(res.status_code == 200), res.status_code

In [ ]:
from sectiontally import SectionTally
from course import Course

In [ ]:
st = SectionTally()

In [44]:
response = requests.post(url='https://banner9.rowan.edu/ords/ssb/bwckctlg.p_disp_course_detail',
            data= {'cat_term_in': term,
                    'subj_code_in': subj,
                    'crse_numb_in': crse})

assert (response.status_code == 200), f"error: {response.status_code}, unable to find course"

soup = BeautifulSoup(response.text, features="html.parser")